**17. Para aquel contenido georeferenciado publicado anónimamente indicar por país, cuántas IPs de usuarios corresponden a IPv4 y cuantas a IPv6**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Bajo el csv **contents** y **geo_tags**

In [ ]:
downloaded = drive.CreateFile({'id':"1SnCVFJvTsaEPrmh-PpVGH6-toBVsD6NL"})
downloaded.GetContentFile('contents.csv') 

downloaded = drive.CreateFile({'id':"1qLu79VkVJxWp9i4Pnq9cxSLjW6Nipxp9"})
downloaded.GetContentFile('geo_tags.csv') 

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Leo los CSVs:

In [ ]:
sqlContext = SQLContext(sc)
contents = sqlContext.read.csv('contents.csv', header=True, inferSchema=True)
rdd_contents = contents.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id='10', namespace='0', revision_id='137966826', parent_revision='137825165', revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id='4642283', revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id='15', namespace='0', revision_id='137986788', parent_revision='133666383', revision_timestamp='2021-08-29T17:16:00Z',

In [ ]:
sqlContext = SQLContext(sc)
geo = sqlContext.read.csv('geo_tags.csv', header=True, inferSchema=True)
rdd_geo = geo.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd_geo.take(5)

[Row(gt_id='158041', gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844381', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.70199966, gt_lon=-89.20800018, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844383', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.39999962, gt_lon=-89.75, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844388', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=12.85000038, gt_lon=-88.0, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

Solo nos interesan los contenidos que están georeferenciados y los públicados anonimamente. 

Contents tiene una columna que es "revisor_ip" la cual indica: IP del revisor (en caso de que **no estuviera registrado**). Entonces los contenidos que tengan una IP cargada en este campo serán los públicados anonimamente, ya que son de usuarios NO registrados.


In [ ]:
rdd_contents.filter(lambda x: x.id != None and x.id.isnumeric() and x.revisor_ip != None).take(5)

[Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Commodore Amiga 500', id='46', namespace='0', revision_id='136224900', parent_revision='135798484', revision_timestamp='2021-06-10T05:53:13Z', revisor_username=None, revisor_id=None, revisor_ip='2806:1000:8101:7AFD:4962:AE:51CB:EFFD', revisor_comment=None),
 Row(title='Circo', id='77', namespace='0', revision_id='137649914', parent_revision='137435739', revision_timestamp='2021-08-13T18:02:53Z', revisor_username=None, revisor_id=None, revisor_ip='190.164.54.234', revisor_comment='/* top */añadí chile a la lista de animales prohibidos'),
 Row(title='Agujero negro de Schwarzschild', id='81', namespace='0', revision_id='129573228', parent_revision='129573224', revision_timestamp='2020-09-26T16:21:09Z', revisor

Ya con el filtrado anterior, lo único que nos interesa de contents son las ips

In [ ]:
ips = rdd_contents.filter(lambda x: x.id != None and x.id.isnumeric() and x.revisor_ip != None).map(lambda x: (int(x.id),x.revisor_ip))

ips.take(5)

[(7, '2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2'),
 (46, '2806:1000:8101:7AFD:4962:AE:51CB:EFFD'),
 (77, '190.164.54.234'),
 (81, '179.53.218.243'),
 (160, '190.232.74.253')]

Ahora trabajo con geo_tags:

No me interesan los datos que NO tengan el país cargado

In [ ]:
rdd_geo.filter(lambda x: x.gt_page_id != None and x.gt_country != None).take(5)

[Row(gt_id='8381128', gt_page_id=1314296, gt_globe='earth', gt_primary=0, gt_lat=40.94710159, gt_lon=-6.48768377, gt_dim=1000, gt_type='landmark', gt_name=None, gt_country='ES', gt_region=None),
 Row(gt_id='8382536', gt_page_id=1363470, gt_globe='earth', gt_primary=0, gt_lat=11.09305573, gt_lon=-63.96805573, gt_dim=10000, gt_type='mountain', gt_name=None, gt_country='VE', gt_region=None),
 Row(gt_id='26172936', gt_page_id=6619663, gt_globe='earth', gt_primary=1, gt_lat=53.12129974, gt_lon=-3.20099998, gt_dim=10000, gt_type='mountain', gt_name=None, gt_country='GB', gt_region=None),
 Row(gt_id='38930505', gt_page_id=2803412, gt_globe='earth', gt_primary=0, gt_lat=-27.53333282, gt_lon=-64.96666718, gt_dim=10000, gt_type='waterbody', gt_name=None, gt_country='AR', gt_region=None),
 Row(gt_id='41226039', gt_page_id=2803412, gt_globe='earth', gt_primary=0, gt_lat=-38.75444444, gt_lon=-62.95611111, gt_dim=1000, gt_type=None, gt_name=None, gt_country='AR', gt_region=None)]

In [ ]:
paises = rdd_geo.filter(lambda x: x.gt_page_id != None and x.gt_country != None).map(lambda x: (x.gt_page_id, x.gt_country))

paises.take(5)

[(1314296, 'ES'),
 (1363470, 'VE'),
 (6619663, 'GB'),
 (2803412, 'AR'),
 (2803412, 'AR')]

Entonces ahora hago un join para quedarme con los contenidos georeferenciados:

In [ ]:
cont_geo = ips.join(paises)

cont_geo.take(10)

[(8330, ('2800:A4:3143:5200:F45E:57A4:64B5:AA67', 'US')),
 (63777, ('79.191.75.161', 'FR')),
 (111251, ('168.205.135.225', 'CL')),
 (185612, ('212.128.102.127', 'FR')),
 (197883, ('188.26.215.120', 'GB')),
 (217245, ('24.55.160.192', 'US')),
 (236033, ('181.9.148.254', 'DO')),
 (250656, ('217.216.195.68', 'TR')),
 (251181, ('181.9.174.19', 'US')),
 (315350, ('2800:E2:1B00:35D9:6C37:96B0:222D:DCBA', 'GQ'))]

Para diferenciar las ips lo voy a hacer a partir de las longitudes.

Las IPv4 como máximo pueden tener 15 de longitud: XXX.XXX.XXX.XXX

In [ ]:
cont_geo.map(lambda x: ((x[1][1], 'IPv4' if len(x[1][0]) < 15 else 'IPv6'),1)).take(5)

[(('US', 'IPv6'), 1),
 (('FR', 'IPv4'), 1),
 (('CL', 'IPv6'), 1),
 (('FR', 'IPv6'), 1),
 (('GB', 'IPv4'), 1)]

Finalmente acá tenemos lo pedido:

In [ ]:
cont_geo.map(lambda x: ((x[1][1], 'IPv4' if len(x[1][0]) < 15 else 'IPv6'),1)).countByKey()

defaultdict(int,
            {('AE', 'IPv4'): 1,
             ('AE', 'IPv6'): 1,
             ('AF', 'IPv4'): 1,
             ('AQ', 'IPv4'): 5,
             ('AQ', 'IPv6'): 1,
             ('AR', 'IPv4'): 44,
             ('AR', 'IPv6'): 13,
             ('AT', 'IPv4'): 6,
             ('AT', 'IPv6'): 2,
             ('AU', 'IPv4'): 4,
             ('BE', 'IPv4'): 4,
             ('BE', 'IPv6'): 2,
             ('BO', 'IPv4'): 8,
             ('BO', 'IPv6'): 13,
             ('BR', 'IPv4'): 2,
             ('BR', 'IPv6'): 2,
             ('BS', 'IPv4'): 1,
             ('BS', 'IPv6'): 1,
             ('BZ', 'IPv4'): 1,
             ('CA', 'IPv4'): 9,
             ('CF', 'IPv4'): 1,
             ('CH', 'IPv6'): 3,
             ('CL', 'IPv4'): 3,
             ('CL', 'IPv6'): 52,
             ('CN', 'IPv4'): 2,
             ('CN', 'IPv6'): 2,
             ('CO', 'IPv4'): 24,
             ('CR', 'IPv4'): 37,
             ('CR', 'IPv6'): 4,
             ('CU', 'IPv4'): 2,
             ('CZ